In [ ]:
//Import para Jupyter-notebooks 
import $ivy.`org.apache.spark::spark-sql:2.4.0`
import $ivy.`sh.almond::almond-spark:0.6.0`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

---
## Join entre tablas temporales

1. Definir la sesión de Spark

In [ ]:
import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

2. Crear un dataframe desde un archivo plano ubicación: resources/Impuesto_de_Transporte_-_MinEnerg_a.csv y resources/Departamentos_y_municipios_de_Colombia.csv

In [ ]:
import spark.implicits._
val impuestoDf = ???
val municipios = ???

In [ ]:
val impuestoDf = spark.read.format("csv").option("header",true).load("resources/Impuesto_de_Transporte_-_MinEnerg_a.csv")
val municipios = spark.read.format("csv").option("header",true).load("resources/Departamentos_y_municipios_de_Colombia.csv")

3. Registra el dataframe como una tabla temporal y envia ejecuta la sentencia sql

In [ ]:
val query = """
    select * from impuestos i inner join municipios m on i.municipio = m.municipio
"""
impuestoDf._______("impuestos")
municipios._______("municipios")
spark.________

In [ ]:
val query = """
    select * from impuestos i inner join municipios m on i.municipio = UPPER(m.municipio)
"""
impuestoDf.createOrReplaceTempView("impuestos")
municipios.createOrReplaceTempView("municipios")
spark.sql(query).show(5)

4. El resultado no es lo esperado? Revisa las llaves con las que se esta haciendo el Join y busca una función para que funcione.

In [ ]:
val query = """
    select * from impuestos i inner join municipios m on i.municipio = UPPER(m.municipio)
"""

5. Revisa con printSchema los nombres de la columna del dataframe de municipios, 

In [ ]:
municipios.printSchema

Cambia el nombre de las columnas y haz que el join contenga: {ano, trimestre,Código dane dep, departamento, Código dane municipio, municipio, total impuesto}

In [ ]:
val newMunicipios: DataFrame = ???

newMunicipios.printSchema
newMunicipios.createOrReplaceTempView("municipios")

val query = ???
spark._____

In [ ]:
val newMunicipios: DataFrame = municipios.withColumnRenamed("REGION", "region")
.withColumnRenamed("CÓDIGO DANE DEL DEPARTAMENTO", "codDaneDep")
.withColumnRenamed("DEPARTAMENTO", "departamento")
.withColumnRenamed("CÓDIGO DANE DEL MUNICIPIO", "codDaneMunicipio")
.withColumnRenamed("MUNICIPIO","municipio")

// Otra alternativa 
//val newColumns = Seq("region","codDaneDep","departamento", "codDaneMunicipio", "municipio")
//val newMunicipios = municipios.toDF(newColumns:_*)

newMunicipios.printSchema
newMunicipios.createOrReplaceTempView("municipios")

val query = """
    select i.ano, i.trimestre,m.codDaneDep, m.departamento, m.codDaneMunicipio, m.municipio, i.totalimpuesto 
    from impuestos i inner join municipios m on i.municipio = UPPER(m.municipio)
"""
spark.sql(query).show(5)